# Make Synthetic Genome
Author : Mathieu Giguere \
Date : 02/07/2024 \
Brief : Strings together the genes of interest of some species into a singular synthetic genome. \
Dependencies : 

## plan

### 1. Lire les fichiers genomes (fasta) -> big string
### 2. Lire les fichiers gffs -> Extraire les coordonnees des genes d'interets
### 3. Pour chaque genome, chercher la sous-liste des bases de chaque gene d'interet et les appends avec un spacer 'NNNNNNNNNN'
### 4. Append tous les str en 1 genome synthetique avec spacer 'N+' adequat
### 5. Write synthetic genome file in fasta format

In [1]:
import re
import pandas as pd

In [2]:
with open("genes-species.xlsx") as xlsx:
    xlsx_content = xlsx.read()
    
csv = re.sub("\\t", ",", xlsx_content)

new_csv = open("gene-species.csv", "w")
new_csv.write(csv)
new_csv.close()

In [3]:
df = pd.read_csv("gene-species.csv")

df = df.groupby('Species')['Gene ID'].apply(list).reset_index()

df

,Species,Gene ID
0,Aspergillus fumigatus,"[ AFUA_4G06890, AFUA_6G05140, AFUA_2G00320, A..."
1,Candida albicans,"[C5_00660C_A, C1_04770C_A, C1_02420C_A, CR_008..."
2,Candida auris,"[B9J08_001448, B9J08_003737, B9J08_000964, B9..."
3,Candida parapsilosis,"[CPAR2_303740, CPAR2_105550, CPAR2_106400, CPA..."
4,Candida tropicalis,"[CTRG_05283, CTRG_04480, CTRG_04661, CTRG_0268..."
5,Cryptococcus neoformans,"[CNA00300, CNN02320, CNE02100, CNA05950, CNF04..."
6,Nasakeomyces glabrata,"[CAGL0E04334g, CAGL0F01793g, CAGL0G01034g, CA..."
7,Pichia kudriavzevii,"[JL09_g2508, JL09_g3074, JL09_g1956, JL09_g200..."


In [4]:
species_genome_file_list = ["fasta_files/unzipped/Aspergillus_fumigatus.ASM265v1.dna.toplevel.fa",
                           "fasta_files/unzipped/C_albicans_SC5314_version_A22-s07-m01-r195_chromosomes.fasta",
                           "fasta_files/unzipped/C_auris_B8441_version_s01-m03-r08_chromosomes.fasta",
                           "fasta_files/unzipped/C_glabrata_CBS138_version_s05-m03-r06_chromosomes.fasta",
                           "fasta_files/unzipped/C_parapsilosis_CDC317_version_s01-m06-r03_chromosomes.fasta",
                           "fasta_files/unzipped/Candida_tropicalis.GCA000006335v3.dna.primary_assembly.GG692418.fa",
                           "fasta_files/unzipped/Cryptococcus_neoformans.ASM9104v1.dna.toplevel.fa",
                           "fasta_files/unzipped/Pichia_kudriavzevii_gca_000764455.ASM76445v1.dna.toplevel.fa"]

species_list = ["A_fumigatus", "C_albicans", "C_auris", "C_glabrata", "C_parapsilosis", "C_tropicalis", "C_neoformans", "P_kudriavzevii"]

In [5]:
# Define the file's name.
filename = "fasta_files/unzipped/Aspergillus_fumigatus.ASM265v1.dna.toplevel.fa"

# Open the file and read its content.
with open(filename) as f:
    content = f.read()
    
# Removes first line puts a space
x = re.sub(">.*\\n", " ", content)

In [6]:
gff_file = "gff_files/Aspergillus_fumigatus.ASM265v1.59.gff3"

with open(gff_file) as g:
    gff_content = g.read()

# find gene coordinates
y = re.findall("gene.*ID=gene:AFUA_6G12400", gff_content)
y

['gene\t3126474\t3132292\t.\t-\t.\tID=gene:AFUA_6G12400']

In [7]:
coordinates = re.findall("[0-9]+", y[0])
print(coordinates)
begin = int(coordinates[0])
print(begin)
end = int(coordinates[1])
print(end)

['3126474', '3132292', '6', '12400']
3126474
3132292


In [9]:
gene_coor = x[begin:end]

gene = re.sub("\\n", "", gene_coor)